In [1]:
import os
CURRENT_PATH = os.getcwd()

import numpy as np

import sklearn
import torch
from torch.nn import functional as F
import torch.optim as optim

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [2]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.Conv = torch.nn.Conv2d(in_channels=1, out_channels=1 ,kernel_size=3, padding=1)
        self.BN = torch.nn.BatchNorm2d(num_features=1)
        self.Pool = torch.nn.MaxPool2d(kernel_size=3, stride=1, padding=1)

        self.NN1 = torch.nn.Linear(1600, 400)
        self.NN2 = torch.nn.Linear(400, 128)
        self.NN3 = torch.nn.Linear(128, 64)
        self.NN4 = torch.nn.Linear(64, 2)


    def forward(self, input):
        x = self.Conv(input)
        x = self.BN(x)
        x = F.relu(x)
        x = self.Pool(x)
        x = F.dropout(x, p=0.2)
        x = torch.flatten(input, start_dim=1)

        x = self.NN1(x)
        x = F.relu(x)
        x = self.NN2(x)
        x = F.relu(x)
        x = self.NN3(x)
        x = F.relu(x)
        x = self.NN4(x)

        x[:, 0] = torch.tanh(x[:, 0])*150 + 50
        x[:, 1] = torch.tanh(x[:, 1])*30

        return x


model = CNN()
model.load_state_dict(torch.load('./paramet.pt', map_location='cpu'))
model.eval()

CNN(
  (Conv): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (BN): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (Pool): MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
  (NN1): Linear(in_features=1600, out_features=400, bias=True)
  (NN2): Linear(in_features=400, out_features=128, bias=True)
  (NN3): Linear(in_features=128, out_features=64, bias=True)
  (NN4): Linear(in_features=64, out_features=2, bias=True)
)

In [4]:
import pygame
import matplotlib.pyplot as plt
import numpy as np

CURRENT_PATH = os.getcwd()

from olympics_engine.scenario import wrestling
from olympics_engine.generator import create_scenario
gamemap =  create_scenario('wrestling')        #load map config
game = wrestling(gamemap)

print('action dim 0 range = ', game.action_f)
print('action dim 1 range = ', game.action_theta)
our_team = 0        #we control team_0
obs = game.reset()
done = False
RENDER = False
if RENDER:
    game.render()

next_obs = obs
while not done:
    input = np.array([np.expand_dims(obs[0]['agent_obs'], axis=0)]).astype(np.float32)
    input = torch.tensor(input)
    #input = torch.tensor([[obs[0]['agent_obs']]], dtype=torch.float32)
    output = model(input)
    #print(output)
    action_team_0 = output[0]    #for now we use random actions for both teams
    #action_team_0 = [np.random.uniform(-100,200), np.random.uniform(-30, 30)]
    action_team_1 = [np.random.uniform(-100,200), np.random.uniform(-30, 30)]
    
    if our_team:
        input_action = [action_team_1, action_team_0]
    else:
        input_action = [action_team_0, action_team_1]


    next_obs, reward, done, _ = game.step(input_action)

    if RENDER:
        game.render()
    
    obs = next_obs
print('final reward = ', reward)
pygame.display.quit()
pygame.quit()

action dim 0 range =  [-100, 200]
action dim 1 range =  [-30, 30]
final reward =  [1.0, 0]


In [ ]:
import pygame
import matplotlib.pyplot as plt

CURRENT_PATH = os.getcwd()

from olympics_engine.scenario import wrestling
from olympics_engine.generator import create_scenario
gamemap =  create_scenario('wrestling')        #load map config
game = wrestling(gamemap)

print('action dim 0 range = ', game.action_f)
print('action dim 1 range = ', game.action_theta)
our_team = 0        #we control team_0
obs = game.reset()
done = False
RENDER = False
if RENDER:
    game.render()

next_obs = obs

obs = np.array([np.expand_dims(obs[0]['agent_obs'], axis=0)]).astype(np.float32)
obs = torch.tensor(obs)
print(obs.shape)

out = model(obs)
print(out)